# install deps

In [ ]:
import os

# Detect if we are in the TIRA sandbox
# Install the required dependencies if we are not in the sandbox.
if 'TIRA_DATASET_ID' not in os.environ:
    !pip3 install python-terrier tira==0.0.88 ir_datasets
else:
    print('We are in the TIRA sandbox.')


# import libs

In [ ]:
# Import the required libraries
print('importing libraries...')
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
ensure_pyterrier_is_loaded()

from load_dataset import load_dataset
from create_index import create_index
from create_model import create_model
from generate_custom_stopwords_improved import generate_custom_stopwords_improved
print('Done. Libraries imported.')

# create run for different stopword sizes

In [ ]:
import glob
from create_index import create_index
from load_dataset import load_dataset
from datetime import datetime
import json

folder_path = './stopwordlists/custom/'
pattern = 'custom_*.txt'
custom_stopword_list = glob.glob(folder_path + pattern)
training_dataset = 'ir-lab-jena-leipzig-wise-2023/training-20231104-training'
load_dataset_result = load_dataset(training_dataset)
queries = load_dataset(training_dataset)['queries']

for improved_stopwords in custom_stopword_list:
    run_name = improved_stopwords.replace("./stopwordlists/custom/", '')
    config = {'stopwords': improved_stopwords, 'stemmer': None}
    print("RUN FOR STOPWORDS:", run_name)

    improved_index = create_index(load_dataset(training_dataset)['documents'], config)
    print("index created")

    improved_model = create_model(improved_index)
    print("model created")

    run = improved_model(queries)

    output_dir = 'runs/training/improved-stopwords'
    run_output_dir = output_dir + '/' + run_name

    !rm -Rf {run_output_dir}
    !mkdir -p {run_output_dir}

    persist_and_normalize_run(run, run_name, run_output_dir)

# compare to standard english stopword list

In [ ]:
from trectools import TrecRun, TrecQrel, TrecEval
from tira.rest_api_client import Client
from glob import glob
import pandas as pd
tira = Client()

folder_path = 'runs/training/improved-stopwords/'
pattern = '**/run.txt'
files = glob(folder_path + pattern)


def load_qrels(dataset):
    return TrecQrel(tira.download_dataset('ir-lab-jena-leipzig-wise-2023', dataset, truth_dataset=True) + '/qrels.txt')

def evaluate_run(qrels, runFile):
    run = TrecRun(runFile)
    trec_eval = TrecEval(run, qrels)

    return {
        'run': run.get_runid(),
        'nDCG@10': trec_eval.get_ndcg(depth=10),
        'nDCG@10 (unjudgedRemoved)': trec_eval.get_ndcg(depth=10, removeUnjudged=True),
        'MAP': trec_eval.get_map(depth=10),
        'MRR': trec_eval.get_reciprocal_rank(),
        'P@10': trec_eval.get_precision(depth=10),
        'P': trec_eval.get_precision()
    }

def test_model(runFile):
    training_qrels = load_qrels('training-20231104-training')

    print("Overall performance:\n")
    print(evaluate_run(training_qrels, runFile))
    print("\n")

result = []

for file in files:
    print("file: ", file)
    result += [evaluate_run(load_qrels('training-20231104-training'), file)]
    print("-----------------DONE-----------------")

print("Reference:")
reference_file = 'runs/standard_stopwords/run.txt'
result += [evaluate_run(load_qrels('training-20231104-training'), reference_file)]

df = pd.DataFrame(result)
df.sort_values('nDCG@10', ascending=False)


No settings given in /root/.tira/.tira-settings.json. I will use defaults.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.
file:  runs/training/improved-stopwords/custom_8.txt/run.txt


/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


Data file not set yet


/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


-----------------DONE-----------------
file:  runs/training/improved-stopwords/custom_2.txt/run.txt


/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


Data file not set yet
